In [2]:
!pip install jupyter-dash

   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   --------- ------------------------------ 1.8/7.9 MB 10.1 MB/s eta 0:00:01
   ------------------ --------------------- 3.7/7.9 MB 9.9 MB/s eta 0:00:01
   -------------------------------------- - 7.6/7.9 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 7.9/7.9 MB 11.1 MB/s eta 0:00:00

   ---- -----------------------------------  1/10 [Werkzeug]
   ---- -----------------------------------  1/10 [Werkzeug]
   -------------------- -------------------  5/10 [ansi2html]
   ---------------------------- -----------  7/10 [flask]
   -------------------------------- -------  8/10 [dash]
   -------------------------------- -------  8/10 [dash]
   -------------------------------- -------  8/10 [dash]
   -------------------------------- -------  8/10 [dash]
   -------------------------------- -------  8/10 [dash]
   ---------------------------------------- 10/10 [jupyter-dash]



In [1]:
# Import required libraries
import pandas as pd
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Load the SpaceX dataset from URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)

# Min and max payload for slider
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

# Create Dash app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1(
        'SpaceX Launch Records Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}
    ),
    
    html.Br(),
    
    # TASK 1 - Dropdown for Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] +
                [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder="Select a launch site",
        searchable=True
    ),
    
    html.Br(),
    
    # TASK 2 - Pie chart
    html.Div(dcc.Graph(id='success-pie-chart')),
    
    html.Br(),
    
    # TASK 3 - Payload range slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        marks={int(i): str(int(i)) for i in range(int(min_payload), int(max_payload)+1, 2000)},
        value=[min_payload, max_payload]
    ),
    
    html.Br(),
    
    # TASK 4 - Scatter chart
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])

# TASK 2 (CONT) - Callback for Pie Chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        success_counts = spacex_df[spacex_df['class']==1].groupby('Launch Site')['class'].count().reset_index()
        fig = px.pie(
            success_counts,
            values='class',
            names='Launch Site',
            title='Total Successful Launches by Site'
        )
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        outcomes = filtered_df['class'].value_counts().reset_index()
        outcomes.columns = ['class', 'count']
        fig = px.pie(
            outcomes,
            values='count',
            names='class',
            title=f'Success vs. Failure for {selected_site}'
        )
    return fig

# TASK 4 (CONT) - Callback for Scatter Chart
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]
    
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=f'Payload vs. Launch Outcome for {"All Sites" if selected_site=="ALL" else selected_site}',
        hover_data=['Launch Site', 'Payload Mass (kg)']
    )
    return fig

# Run the app inline in JupyterLab/Notebook
app.run(mode='inline')
